# Load Dataset

In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
dataset_train = pd.read_csv("../input/gooogle-stock-price/Google_Stock_Price_Train.csv")
data = dataset_train.iloc[:,0:6].values

In [ ]:
x_train = []
y_train = []
days = 601
i = days
while i <= 1257:
    t = data[i-days:i-1,1:5]
    x_train.append(t)
    y_train.append(data[i,1:5])
    i = i+1
    
for x in x_train:
    for xx in x:
        if type(xx[3]) == str:
            xx[3] = float(xx[3].replace(',',''))
        
for y in y_train:
    if type(y[3]) == str:
        y[3] = float(y[3].replace(',',''))

In [ ]:
x_train = np.array(x_train,dtype = np.float64)[:,:,0:1]
y_train = np.array(y_train,dtype = np.float64)[:,0:1]

# Define Model

In [ ]:
import torch

sig = lambda x: 1/(1+np.exp(-x))
siginv = lambda y: np.log(y/(1-y))

sig = lambda x: x/1000
siginv = lambda y: y*1000

class RNN(torch.nn.Module):
    def __init__(self,in_dim,hidden_dim,out_dim):
        super(RNN,self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hidden_dim = hidden_dim
        self.input_layer = torch.nn.Linear(in_dim, hidden_dim, bias=True)
        self.iteration_layer = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.output_layer = torch.nn.Linear(hidden_dim, out_dim, bias=True)
        self.activation = torch.nn.Tanh()
    
    def forward(self,X):
        relu = torch.nn.ReLU()
        S = []
        M = []
        N = []
        m = torch.min(X[0:1])
        n = torch.max(X[0:1])
        x = (X[0:1]-m)/(n-m)
        S.append(self.input_layer(self.activation(x)))
        M.append(torch.unsqueeze(m,0))
        N.append(torch.unsqueeze(n,0))
        
        for i in range(1,len(X)):
            x = X[i:i+1]

            m = torch.min(x)
            n = torch.max(x)
            
            x = (x-m)/(n-m)
            S.append(self.activation(relu(self.iteration_layer(S[i-1])) + relu(self.input_layer(x))))
            M.append(torch.unsqueeze(m,0))
            N.append(torch.unsqueeze(n,0))
        s = torch.cat(S,0)
        m = torch.cat(M,0)
        n = torch.cat(N,0)
        M = []
        N = []
        for i in range(self.out_dim):
            M.append(torch.unsqueeze(m,1))
        m = torch.cat(M,1)
        for i in range(self.out_dim):
            N.append(torch.unsqueeze(n,1))
        n = torch.cat(N,1)
        y = self.output_layer(s)
        y = y*(n-m)+m
        y = relu(y)
        return y
    
    '''def forward(self,X):
        S = []
        relu = torch.nn.ReLU()
        S.append(self.input_layer((X[0:1])))
        for i in range(1,len(X)):
            x = X[i:i+1]
            S.append(self.activation(relu(self.iteration_layer(S[i-1])) + relu(self.input_layer(x))))
        s = torch.cat(S,0)
        y = relu(self.output_layer(s))
        return y'''
    
    def train(self,x_train,y_train,epoch,rate = 0.01):
        x = np.array(x_train.reshape([-1,self.in_dim]))
        y = np.array(y_train)
        
        X = torch.from_numpy(sig(x)).float()
        Y = torch.from_numpy(sig(y)).float()
        
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        for i in range(epoch):
            optimizer.zero_grad()
            Yhat = self.forward(X)
            loss = criterion(Yhat,Y)
            loss.backward()
            optimizer.step()
            print(i,loss.data.numpy())
            
    def predict(self,x):
        x_1 = x.reshape([-1,self.in_dim])
        X = torch.from_numpy(sig(x_1)).float()
        Y = self.forward(X)
        y = siginv(Y.data.numpy()[0])
        return y

# Train

In [ ]:
M = RNN((days-1)*1,100,1)

In [ ]:
M.train(x_train,y_train,30,1)
M.train(x_train,y_train,30,0.1)

# Prediction

## Prediction without data update

In [ ]:
dataset_test = pd.read_csv("../input/gooogle-stock-price/Google_Stock_Price_Test.csv")
test_data = dataset_train.iloc[:,1:2].values
for i in range(len(test_data)):
    for j in range(len(test_data[0])):
        if type(test_data[i,j]) == str:
            test_data[i,j] = float(test_data[i,j].replace(',',''))
test_data = np.array(test_data,dtype = np.float32)

n = len(test_data)
k = 0

prediction = np.array(test_data)
prediction[k:k+days-1] = test_data[k:k+days-1]
for i in range(k+days-1,n):
    prediction[i] = M.predict(prediction[i-(days-1):i])

import matplotlib.pyplot as plt
a,b = test_data[:n],prediction[:n]
plt.plot(test_data[:n])
plt.plot(prediction[:n])

# Prediction with data update

In [ ]:
dataset_test = pd.read_csv("../input/gooogle-stock-price/Google_Stock_Price_Test.csv")
test_data = dataset_train.iloc[:,1:2].values
for i in range(len(test_data)):
    for j in range(len(test_data[0])):
        if type(test_data[i,j]) == str:
            test_data[i,j] = float(test_data[i,j].replace(',',''))
test_data = np.array(test_data,dtype = np.float32)

n = len(test_data)

prediction = np.zeros(test_data.shape)
prediction[0:days-1] = test_data[0:days-1]
for i in range(days-1,n):
    prediction[i] = M.predict(test_data[i-(days-1):i])

import matplotlib.pyplot as plt
a,b = test_data[:n],prediction[:n]
plt.plot(test_data[:n])
plt.plot(prediction[:n])